![logo](IPUpy.png)

In [122]:
print('Hello world')

-------[5888x]-------
Hello world


In [123]:
print('I am tile', __tileid % 5)

-------[1178x]-------
I am tile 0

-------[1178x]-------
I am tile 1

-------[1178x]-------
I am tile 2

-------[1177x]-------
I am tile 3

-------[1177x]-------
I am tile 4


# Word Search

In [2]:
words = __tiledata.splitlines()
# print(words)

In [3]:
print([w for w in words if 'ipu' in w])

-------[1x]-------
['Itaipu']

-------[1x]-------
['Jaipur', "Jaipur's"]

-------[1x]-------
['Lilliput', "Lilliput's", 'Lilliputian', "Lilliputian's", 'Lilliputians']

-------[1x]-------
['Oedipus', "Oedipus's"]

-------[1x]-------
['manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', "manipulation's", 'manipulations', 'manipulativ']

-------[1x]-------
['manipulator', "manipulator's", 'manipulators']

-------[1x]-------
['multipurpose']

-------[1x]-------
['stipulate', 'stipulated', 'stipulates', 'stipulating', 'stipulation', "stipulation's", 'stipulations']

-------[5880x]-------
[]


# MandleBrot

In [126]:
# Setup
H, W = 64, 92
X, Y = __tileid % W, __tileid // W

x_start = (X / W) * 4.94 - 3
y_start = (Y / H) * 4.48 - 2.24
x_end = (X / W) * 0.014 - 0.632
y_end = (Y / H) * 0.012 - 0.474

max_iter_start = 18

def setColour(t):
    T = 0.5
    if t < T:
        val = 256 * t / T
        return setPixel(0, 255 - val, 255)
    val = 256 * (t - T) / (1 - T)
    return setPixel(val, 0, 255 - val)

In [128]:
x0, y0, max_iteration = x_start, y_start, max_iter_start

!RepeatBlock(900)

x, y, iteration = 0, 0, 0
while ((x*x + y*y) <= 4) & (iteration < max_iteration):
    x, y = x*x - y*y + x0, 2*x*y + y0
    iteration += 1
setColour(iteration / max_iteration)

x0 += (x_end - x0) * 0.01
y0 += (y_end - y0) * 0.01
max_iteration *= 1.001

# Black Hat Snek
It's easy to exhaust a 2<sup>20</sup> search space is when you have 2<sup>14</sup> python interpreters

In [120]:
def Xcrypt(key, data):
    return bytes(d ^ (((key % 0xfffff) >> (8 * (i % 3))) & 0xff) for i, d in enumerate(data))

secret_message = Xcrypt(
    key=314159,
    data=b"IPUs are now available to use for free on paperspace!", 
)

In [121]:
for key in range(__tileid, 1 << 20, 5888):
    result = Xcrypt(key, cyphertext)
    for crib in ('Graphcore', 'IPU', 'POD4'):
        if crib in result:
            print("Hit:", result)

-------[1x]-------
Hit: b'IPUs are now available to use for free on paperspace!'

-------[1x]-------
Hit: b'oIPU9dT|%Hvr\x06xsGpiG{iC9qI9pU|%@vw\x06\x7fwC|%Iw%VxuCkvVxfC8'


In [ ]:
def checkPwd(pwd):
    if pwd == 'Jaipur':
        